In [1]:
# Packages laden
import numpy as np
import pandas as pd
import math
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report,confusion_matrix,auc, log_loss 
from pandas import DataFrame
from keras.backend import binary_crossentropy
import tensorflow as tf
from pandas import concat

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Laden der Daten
DATA_FILE="Dropbox/Masterarbeit/Masterarbeit/Datensätze/Geburtenrate/Datensatz.xlsx"
df = pd.read_excel(DATA_FILE)
df=pd.read_excel(DATA_FILE, header=0, parse_dates=[0], index_col='Jahr-Monat',squeeze=True)

In [5]:
# Ein array mit allen Einträgen --> zur besseren Weiterverarbeitung
ZR=df.values

In [6]:
# Skalierung [-1,1]
def scaling(dataset,l_bound=-1, h_bound=1):
    dataset = dataset.reshape(len(dataset), 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(dataset)
    scaled_dataset = scaler.transform(dataset)
    return scaled_dataset

# Umkehrung der Skalierung
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [7]:
ZR_N=scaling(ZR)

# Anteil der Datensätze am Gesamtdatensatz
ZR_100=ZR
ZR_46=ZR[-400:]
ZR_12=ZR[-100:]

# Unterteilung der Teildatensätze in Trainings-und Testdaten
ZR_100_TR, ZR_100_TE= ZR_100[0:605], ZR_100[605:len(ZR_100)]
ZR_46_TR, ZR_46_TE= ZR_46[0:250], ZR_46[250:len(ZR_46)]
ZR_12_TR, ZR_12_TE= ZR_12[0:50], ZR_12[50:len(ZR_12)]

# Anteil der Datensätze am Gesamtdatensatz: skalierte Daten
ZR_N100=ZR_N
ZR_N46=ZR_N[-400:]
ZR_N12=ZR_N[-100:]

# Unterteilung der Teildatensätze in Trainings-und Testdaten
ZR_N100_TR, ZR_N100_TE= ZR_N100[0:605], ZR_N100[605:len(ZR_N100)]
ZR_N46_TR, ZR_N46_TE= ZR_N46[0:250], ZR_N46[250:len(ZR_N46)]
ZR_N12_TR, ZR_N12_TE= ZR_N12[0:50], ZR_N12[50:len(ZR_N12)]

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
# Funktion zur Erzeugung von y zur Trendvorhersage/Klassifikation t+1: y=(-1,0,1)
def create_Y_trend(dataset):
    dataY=[0]*(len(dataset)-1)
    for i in range(len(dataset)-1):
        a = dataset[i]-dataset[i+1]
        if a<0:
            dataY[i]=1
        elif a>0:
            dataY[i]=-1
        else:
            dataY[i]=0
    return np.array(dataY)

In [9]:
# Funktion zur Erzeugung von x (Input) mit look_back (Anzahl der verwendeten Vergangenheitswerte)
def X_to_supervised_T(dataset, look_back=1, look_forward=1):
    df = DataFrame(dataset)
    columns = [df.shift(look_forward-1+i) for i in range(1, look_back+1)]
    df = concat(columns, axis=1)
    df = df.drop(0)
    DF=df[::-1]
    df=DF[:len(df)-(look_back+look_forward-2)]
    DF=df[::-1]
    return DF

In [10]:
# Festlegung: look_back, look_forward + Anwendung der Funktion
# Inputneuronen: 12
# Vorhersagehorizont: t+1
look_back=12
look_forward=1

# Erzeugen von y 

# Dateninput 100%
# Skalierte Daten
trainY_N100=create_Y_trend(ZR_N100_TR)
testY_N100=create_Y_trend(ZR_N100_TE)

# Original Daten
trainY_100=create_Y_trend(ZR_100_TR)
testY_100=create_Y_trend(ZR_100_TE)

# Dateninput 46%
# Skalierte Daten
trainY_N46=create_Y_trend(ZR_N46_TR)
testY_N46=create_Y_trend(ZR_N46_TE)

# Original Daten
trainY_46=create_Y_trend(ZR_46_TR)
testY_46=create_Y_trend(ZR_46_TE)

# Dateninput 12%
# Skalierte Daten
trainY_N12=create_Y_trend(ZR_N12_TR)
testY_N12=create_Y_trend(ZR_N12_TE)

# Original Daten
trainY_12=create_Y_trend(ZR_12_TR)
testY_12=create_Y_trend(ZR_12_TE)

In [11]:
# Erzeugen von X 

# Dateninput: 100%
# Skalierte Daten
trainX_N100=X_to_supervised_T(ZR_N100_TR,look_back,look_forward)
testX_N100=X_to_supervised_T(ZR_N100_TE,look_back,look_forward)

#Originaldaten
trainX_100=X_to_supervised_T(ZR_100_TR,look_back,look_forward)
testX_100=X_to_supervised_T(ZR_100_TE,look_back,look_forward)

# Dateninput: 46%
# Skalierte Daten
trainX_N46=X_to_supervised_T(ZR_N46_TR,look_back,look_forward)
testX_N46=X_to_supervised_T(ZR_N46_TE,look_back,look_forward)

# Originaldaten
trainX_46=X_to_supervised_T(ZR_46_TR,look_back,look_forward)
testX_46=X_to_supervised_T(ZR_46_TE,look_back,look_forward)

# Dateninput: 12%
# Skalierte Daten
trainX_N12=X_to_supervised_T(ZR_N12_TR,look_back,look_forward)
testX_N12=X_to_supervised_T(ZR_N12_TE,look_back,look_forward)

#Originaldaten
trainX_12=X_to_supervised_T(ZR_12_TR,look_back,look_forward)
testX_12=X_to_supervised_T(ZR_12_TE,look_back,look_forward)

In [12]:
# Zuschneiden der Datensätze --> gleiche Länge von X und y

# Dateninput: 100%
# Skalierte Daten
# Trainingsdaten
trainY_N100=trainY_N100.tolist()
trainY_N100=trainY_N100[look_back-1:]
trainY_N100=np.array(trainY_N100)
print(trainX_N100.shape, trainY_N100.shape)
# Testdaten
testY_N100=testY_N100.tolist()
testY_N100=testY_N100[look_back-1:]
testY_N100=np.array(testY_N100)
print(testX_N100.shape, testY_N100.shape)

# Originaldaten
# Trainingsdaten
trainY_100=trainY_100.tolist()
trainY_100=trainY_100[look_back-1:]
trainY_100=np.array(trainY_100)
print(trainX_100.shape, trainY_100.shape)
# Testdaten
testY_100=testY_100.tolist()
testY_100=testY_100[look_back-1:]
testY_100=np.array(testY_100)
print(testX_100.shape, testY_100.shape)

(593, 12) (593,)
(246, 12) (246,)
(593, 12) (593,)
(246, 12) (246,)


In [31]:
# Dateninput: 46%
# Skalierte Daten
# Trainingsdaten
trainY_N46=trainY_N46.tolist()
trainY_N46=trainY_N46[look_back-1:]
trainY_N46=np.array(trainY_N46)
print(trainX_N46.shape, trainY_N46.shape)
# Testdaten
testY_N46=testY_N46.tolist()
testY_N46=testY_N46[look_back-1:]
testY_N46=np.array(testY_N46)
print(testX_N46.shape, testY_N46.shape)

# Originaldaten
# Trainingsdaten
trainY_46=trainY_46.tolist()
trainY_46=trainY_46[look_back-1:]
trainY_46=np.array(trainY_46)
print(trainX_46.shape, trainY_46.shape)
# Testdaten
testY_46=testY_46.tolist()
testY_46=testY_46[look_back-1:]
testY_46=np.array(testY_46)
print(testX_46.shape, testY_46.shape)

(238, 12) (238,)
(138, 12) (138,)
(238, 12) (238,)
(138, 12) (138,)


In [47]:
# Dateninput: 12%
# Skalierte Daten

# Trainingsdaten
trainY_N12=trainY_N12.tolist()
trainY_N12=trainY_N12[look_back-1:]
trainY_N12=np.array(trainY_N12)
print(trainX_N12.shape, trainY_N12.shape)
# Testdaten
testY_N12=testY_N12.tolist()
testY_N12=testY_N12[look_back-1:]
testY_N12=np.array(testY_N12)
print(testX_N12.shape, testY_N12.shape)

# Originaldaten
# Trainingsdaten
trainY_12=trainY_12.tolist()
trainY_12=trainY_12[look_back-1:]
trainY_12=np.array(trainY_12)
print(trainX_12.shape, trainY_12.shape)
# Testdaten
testY_12=testY_12.tolist()
testY_12=testY_12[look_back-1:]
testY_12=np.array(testY_12)
print(testX_12.shape, testY_12.shape)

(38, 12) (38,)
(38, 12) (38,)
(38, 12) (38,)
(38, 12) (38,)


In [13]:
# Funktion: X,y trennen, Form von X anpassen, Modell zusammenbauen, Modell anpassen
def fit_lstm(X, y , batch_size, blocks, epochen):
    X=array(X)
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(blocks, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=epochen, batch_size=1)
    return model

In [14]:
# Replizierbarkeit gewährleisten
np.random.seed(0)

In [15]:
# Modell anpassen: 100%
lstm_model_100 = fit_lstm(trainX_N100, trainY_N100, batch_size=1, blocks=4, epochen=30)

Epoch 1/30
593/593 [==============================] - 4s 7ms/step - loss: -1.2425 - acc: 0.0000e+00
Epoch 2/30
593/593 [==============================] - 2s 4ms/step - loss: -0.8055 - acc: 0.0000e+00
Epoch 3/30
593/593 [==============================] - 3s 4ms/step - loss: -0.8632 - acc: 0.0017
Epoch 4/30
593/593 [==============================] - 3s 4ms/step - loss: -0.2194 - acc: 0.0000e+00
Epoch 5/30
593/593 [==============================] - 3s 5ms/step - loss: -0.6561 - acc: 0.0017
Epoch 6/30
593/593 [==============================] - 3s 4ms/step - loss: -0.1361 - acc: 0.0185
Epoch 7/30
593/593 [==============================] - 3s 4ms/step - loss: -0.8480 - acc: 0.0523
Epoch 8/30
593/593 [==============================] - 3s 4ms/step - loss: -0.8177 - acc: 0.0624
Epoch 9/30
593/593 [==============================] - 2s 4ms/step - loss: -0.8574 - acc: 0.0641
Epoch 10/30
593/593 [==============================] - 2s 4ms/step - loss: -0.7765 - acc: 0.0472
Epoch 11/30
593/593 [======

In [16]:
# Funktion zur Vorhersage
def forecast_lstm(model, batch_size, X):
    X=array(X)
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [17]:
# Validierung des Modells anhand der Testdaten
predictions_100 = list()
for i in range(len(testY_N100)):
    # schrittweise Vorhersage
    X=testX_N100
    yhat = forecast_lstm(lstm_model_100, 1, X.iloc[i,:])
    # Vorhersage speichern
    predictions_100.append(yhat)
    expected = ZR_N100[len(trainX_N100) + i + 1]
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

Month=1, Predicted=-0.504193, Expected=-0.439354
Month=2, Predicted=-0.551467, Expected=-0.460056
Month=3, Predicted=-0.592440, Expected=-0.493534
Month=4, Predicted=-0.601321, Expected=-0.534703
Month=5, Predicted=-0.609342, Expected=-0.658966
Month=6, Predicted=-0.616786, Expected=-0.659201
Month=7, Predicted=-0.616229, Expected=-0.583381
Month=8, Predicted=-0.599686, Expected=-0.633571
Month=9, Predicted=-0.602841, Expected=-0.550451
Month=10, Predicted=-0.597553, Expected=-0.642593
Month=11, Predicted=-0.589286, Expected=-0.578349
Month=12, Predicted=-0.594116, Expected=-0.534208
Month=13, Predicted=-0.601782, Expected=-0.377953
Month=14, Predicted=-0.604010, Expected=-0.391198
Month=15, Predicted=-0.634867, Expected=-0.393857
Month=16, Predicted=-0.636830, Expected=-0.479872
Month=17, Predicted=-0.649309, Expected=-0.568415
Month=18, Predicted=-0.655468, Expected=-0.504745
Month=19, Predicted=-0.656591, Expected=-0.415289
Month=20, Predicted=-0.639803, Expected=-0.562940
Month=21,

Month=237, Predicted=-0.665512, Expected=-0.724801
Month=238, Predicted=-0.651221, Expected=-0.772019
Month=239, Predicted=-0.632654, Expected=-0.660140
Month=240, Predicted=-0.622844, Expected=-0.594905
Month=241, Predicted=-0.617995, Expected=-0.470485
Month=242, Predicted=-0.618527, Expected=-0.485295
Month=243, Predicted=-0.637589, Expected=-0.487459
Month=244, Predicted=-0.638541, Expected=-0.586067
Month=245, Predicted=-0.657315, Expected=-0.726443
Month=246, Predicted=-0.657083, Expected=-0.697815


In [20]:
# Umwandeln der Werte in Trendzahlen [-1,0,1]
def classification(dataset):
    for i in range(len(dataset)):
        if dataset[i]>= 0.05:
            dataset[i]=1
        elif dataset[i]<= -0.05:
            dataset[i]=-1
        else:
            dataset[i]=0
    return dataset

In [21]:
testPredict=classification(predictions_100)

In [22]:
accuracy_score(testY_N100, testPredict)

0.5063829787234042

In [ ]:
# Replizierbarkeit gewährleisten
np.random.seed(0)

In [39]:
# Modell anpassen: 46%
lstm_model_46 = fit_lstm(trainX_N46, trainY_N46, batch_size=1, blocks=4, epochen=200)

Epoch 1/30
238/238 [==============================] - 3s 13ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 2/30
238/238 [==============================] - 2s 7ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 3/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 4/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 5/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 6/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 7/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 8/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 9/30
238/238 [==============================] - 1s 5ms/step - loss: -0.6772 - acc: 0.0000e+00
Epoch 10/30
238/238 [==============================] - 1s 6ms/step - loss: -0.6772 - acc: 0.0000e+0

In [40]:
# Validierung des Modells anhand der Testdaten
predictions_46 = list()
for i in range(len(testY_N46)):
    # schrittweise Vorhersage
    X=testX_N46
    yhat = forecast_lstm(lstm_model_46, 1, X.iloc[i,:])
    # Vorhersage speicher
    predictions_46.append(yhat)
    expected = ZR_N46[len(trainX_N46) + i + 1]
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

Month=1, Predicted=-0.042473, Expected=-0.549851
Month=2, Predicted=-0.035492, Expected=-0.577071
Month=3, Predicted=-0.045312, Expected=-0.614460
Month=4, Predicted=-0.037227, Expected=-0.727434
Month=5, Predicted=-0.042822, Expected=-0.826641
Month=6, Predicted=-0.068573, Expected=-0.763336
Month=7, Predicted=-0.077787, Expected=-0.754445
Month=8, Predicted=-0.093581, Expected=-0.863248
Month=9, Predicted=-0.098265, Expected=-0.761407
Month=10, Predicted=-0.090132, Expected=-0.797570
Month=11, Predicted=-0.084576, Expected=-0.766752
Month=12, Predicted=-0.080308, Expected=-0.733379
Month=13, Predicted=-0.078095, Expected=-0.646686
Month=14, Predicted=-0.064753, Expected=-0.646556
Month=15, Predicted=-0.065623, Expected=-0.667571
Month=16, Predicted=-0.061947, Expected=-0.778406
Month=17, Predicted=-0.055683, Expected=-0.873677
Month=18, Predicted=-0.082383, Expected=-0.822600
Month=19, Predicted=-0.098245, Expected=-0.807608
Month=20, Predicted=-0.112167, Expected=-0.910831
Month=21,

In [41]:
testPredict=classification(predictions_46)

In [43]:
accuracy_score(testY_N46, testPredict)

0.47101449275362317

In [ ]:
# Replizierbarkeit gewährleisten
np.random.seed(0)

In [53]:
# Modell anpassen: 12%
lstm_model_12 = fit_lstm(trainX_N12, trainY_N12, batch_size=1, blocks=4, epochen=30)

Epoch 1/30
38/38 [==============================] - 2s 60ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 2/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 3/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 4/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 5/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 6/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 7/30
38/38 [==============================] - 0s 6ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 8/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 9/30
38/38 [==============================] - 0s 5ms/step - loss: -0.8483 - acc: 0.0000e+00
Epoch 10/30
38/38 [==============================] - ETA: 0s - loss: -0.4884 - acc: 0.0000e+ - 0s 5ms/step - loss: -0

In [54]:
# Validierung des Modells anhand der Testdaten
predictions_12 = list()
for i in range(len(testY_N12)):
    # schrittweise Vorhersage
    X=testX_N12
    yhat = forecast_lstm(lstm_model_12, 1, X.iloc[i,:])
    # Vorhersage speichern
    predictions_12.append(yhat)
    expected = ZR_N12[len(trainX_N12) + i + 1]
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

Month=1, Predicted=-0.285139, Expected=-0.852167
Month=2, Predicted=-0.272046, Expected=-0.860719
Month=3, Predicted=-0.305247, Expected=-0.791443
Month=4, Predicted=-0.313980, Expected=-0.951348
Month=5, Predicted=-0.337515, Expected=-0.835689
Month=6, Predicted=-0.353198, Expected=-0.850941
Month=7, Predicted=-0.369449, Expected=-0.769307
Month=8, Predicted=-0.348322, Expected=-0.761459
Month=9, Predicted=-0.387104, Expected=-0.575325
Month=10, Predicted=-0.355701, Expected=-0.621421
Month=11, Predicted=-0.329926, Expected=-0.613991
Month=12, Predicted=-0.293716, Expected=-0.733926
Month=13, Predicted=-0.257690, Expected=-0.867941
Month=14, Predicted=-0.255668, Expected=-0.836653
Month=15, Predicted=-0.287477, Expected=-0.749778
Month=16, Predicted=-0.289363, Expected=-0.858581
Month=17, Predicted=-0.324727, Expected=-0.780988
Month=18, Predicted=-0.322205, Expected=-0.787793
Month=19, Predicted=-0.330112, Expected=-0.705819
Month=20, Predicted=-0.315274, Expected=-0.670804
Month=21,

In [56]:
testPredict=classification(predictions_12)

In [57]:
accuracy_score(testY_N12, testPredict)

0.6875